Beginning of additional text processing
=======================================
This is to extract information unrelated to geocoding

In [81]:
import pandas as pd
import numpy as np
import time, re, json, requests, os, nltk
verbose = False

In [82]:
#stemmer to find roots of words. Transformations like "Толстого" and "Толстой" to "толст",
#to find different the same word or idea in different usages and cases.
ru_stemmer = nltk.stem.SnowballStemmer('russian')

In [83]:
import os
def Ensure_dir(d):
    """Makes sure a directory 'd' exists, and if it doesn't, it creates one."""
    if not os.path.exists(d):
        os.makedirs(d)
    return 0

In [84]:
#Orthography update function

import re,json
from lxml import html

with open("../resources/special_cases.json",'r',encoding='utf-8') as fp:
    lookup_dict = json.load(fp)

def ortho_rules(text,lookup_dict):
    """
    Takes a text string and a lookup dict as input
    Removes any lingering html tags
    Breaks the string down into component words with split
    Uses replacements to modernize orthography
    Returns re-assembled text with modern orthography
    """
    if type(text)==str:
        text = html.fromstring(text).text_content()
        textlist = text.split(' ')
        out_list = []
        for word in textlist:
            if word in [',','.',';',':']:
                out_list.append(word)
                continue
            punct = ""
            if len(re.findall(r"([,.;:])$",word))!=0:
                   punct = re.findall(r"([,.;:])$",word)[0]
            word = word.strip(',.;:')
            if word in lookup_dict:
                word = lookup_dict[word]
                out_list.append(word)
            else:
                word = re.sub("[Ъъ]$",'',word)
                word = re.sub(r"^([че]{0,2}[ч]?[вбнр]?[веоаи])з([ПФКТШСЧпфктшсч])",r"\1с\2",word,re.IGNORECASE)
                word = re.sub("аго$","ого",word,re.IGNORECASE)
                word = re.sub("яго$","его",word,re.IGNORECASE)
                word = re.sub("ыя$","ые",word,re.IGNORECASE)
                word = re.sub("iя$","ие",word,re.IGNORECASE)
                word = word.replace('i','и') #english
                word = word.replace('I','И') #english
                word = word.replace('і','и') #cyrillic
                word = word.replace('І','И') #cyrillic
                word = word.replace('Ѣ','Е')
                word = word.replace('ѣ','е')
                word = word.replace('Ѳ','Ф')
                word = word.replace('ѳ','ф')
                word += punct
                out_list.append(word)
        out_str = ' '.join(out_list)
        return out_str
    else:
        return None

In [85]:
cyrillic_translit={u'\u0410': 'A', u'\u0430': 'a',u'\u0411': 'B', u'\u0431': 'b',u'\u0412': 'V', u'\u0432': 'v',u'\u0413': 'G', u'\u0433': 'g',u'\u0414': 'D', u'\u0434': 'd',u'\u0415': 'E', u'\u0435': 'e',u'\u0416': 'Zh', u'\u0436': 'zh',u'\u0417': 'Z', u'\u0437': 'z',u'\u0418': 'I', u'\u0438': 'i',u'\u0419': 'I', u'\u0439': 'i',u'\u041a': 'K', u'\u043a': 'k',u'\u041b': 'L', u'\u043b': 'l',u'\u041c': 'M', u'\u043c': 'm',u'\u041d': 'N', u'\u043d': 'n',u'\u041e': 'O', u'\u043e': 'o',u'\u041f': 'P', u'\u043f': 'p',u'\u0420': 'R', u'\u0440': 'r',u'\u0421': 'S', u'\u0441': 's',u'\u0422': 'T', u'\u0442': 't',u'\u0423': 'U', u'\u0443': 'u',u'\u0424': 'F', u'\u0444': 'f',u'\u0425': 'Kh', u'\u0445': 'kh',u'\u0426': 'Ts', u'\u0446': 'ts',u'\u0427': 'Ch', u'\u0447': 'ch',u'\u0428': 'Sh', u'\u0448': 'sh',u'\u0429': 'Shch', u'\u0449': 'shch',u'\u042a': '"', u'\u044a': '"',u'\u042b': 'Y', u'\u044b': 'y',u'\u042c': "'", u'\u044c': "'",u'\u042d': 'E', u'\u044d': 'e',u'\u042e': 'Iu', u'\u044e': 'iu',u'\u042f': 'Ia', u'\u044f': 'ia',u'\u0462': 'E', u'\u0463': 'e'}

def transliterate(word, translit_table):
    """
    Transliterates 'word' based on the key/value pairs in 'translit_table'
    """
    converted_word = ''
    for char in word:
        transchar = ''
        if char in translit_table:
            transchar = translit_table[char]
        else:
            transchar = char
        converted_word += transchar
    return converted_word

Importing and updating data
===========================
Taking existing dataset, updating it with data from other rounds of text processing.

In [86]:
import collections
def update(d, u):
    """
    Recursively updates d with values from u.
    """
    for k, v in u.items():
        if isinstance(v, collections.Mapping):
            r = update(d.get(k, {}), v)
            d[k] = r
        else:
            d[k] = u[k]
    return d

In [87]:
with open("../output/enhanced_dataset.json","r",encoding='utf-8') as fp:
    db = json.load(fp)

In [88]:
market_df = pd.DataFrame.from_csv("../output/process/no_geo.csv")
db = update(db,json.loads(market_df.to_json(orient='index')))

In [89]:
for i in db:
    txt = db[i]['description_old_orth']
    ftype = db[i]['featuretype_old_orth']
    db[i]['description_new_orth'] = ortho_rules(txt,lookup_dict)
    db[i]['featuretype_new_orth'] = ortho_rules(ftype,lookup_dict)

In [90]:
for k,v in db.items():
    db[k]['name_new_orth_lc'] = transliterate(v['name_new_orth'],cyrillic_translit)
    db[k]['name_old_orth_lc'] = transliterate(v['name_old_orth'],cyrillic_translit)
    db[k]['featuretype_new_orth_lc'] = transliterate(v['featuretype_new_orth'],cyrillic_translit)
    db[k]['featuretype_old_orth_lc'] = transliterate(v['featuretype_old_orth'],cyrillic_translit)

Where are the goods and merchants coming from?
==============================================
This section looks for geographic relationships between the town in the entry and named towns that serve as trade partners.

In [91]:
def ru_lang_to_list(liststr):
    """
    takes a list of strings which may themselves be natural language lists in Russian
    Returns list of items in list strings, flattened
    """
    liststr = [a.split(' и ') for a in liststr]
    liststr = [item for sublist in liststr for item in sublist]
    liststr = [a.split(',') for a in liststr]
    liststr = [item for sublist in liststr for item in sublist]
    liststr = [a.strip() for a in liststr]
    liststr = [a for a in liststr if len(a)!=0]
    return liststr

In [92]:
def word_case_sent_finder(wordlist, case_exp, text):
    """
    Finds all words matching case_exp in sentences containing words in wordlist that appear in text
    """
    sent = nltk.tokenize.sent_tokenize(text)
    sent_tok = [nltk.tokenize.wordpunct_tokenize(s) for s in sent]
    returnlist = []
    for s in sent_tok:
        if any([v in s for v in wordlist]):
            for w in s:
                match = re.match(case_exp,w)
                if match:
                    returnlist.append(match.string)
    return returnlist

In [93]:
for i in db:
    txt = db[i]['description_new_orth']
    merch_adj = {}
    #merch_adj finds sequences of words ending in "ские" appearing in sentences that also contain "купцы," or some variation thereof
    #That is, it finds adjectives describing merchants, which in this text, means describing their geographic origins.
    merch_adj1 = word_case_sent_finder(['купцам','Купцы','купцы','купцами','купцов','купечество'], r"[А-Яа-я]+[цс]кие", txt)
    if len(merch_adj1)!=0:
        merch_adj[1]=merch_adj1
    #merch_adj2 just finds all of the different ways of saying "merchants from nearby towns"
    #by finding all words between "купцы из" and "городов"
    merch_adj2 = re.findall(r"из\s+((?:[а-яА-Я,]+\s+)*?(?:городов|мест|с|;|всякие|всякой|малому|привозят|еженедельно|разной) )",txt)
    merch_adj2 = [a.strip() for a in merch_adj2]
    merch_adj2 = [re.sub(r"(?:с|;|всякие|всякой|малому|привозят|еженедельно|разной)$","",a) for a in merch_adj2]
    merch_adj2 = ru_lang_to_list(merch_adj2)
    merch_adj2 = [re.sub(r"^из ","",a) for a in merch_adj2]
    if len(merch_adj2)!=0:
        merch_adj[2]=merch_adj2
    #merch_adj.extend(merch_adj2)
    #merch_adj3 finds instances of merchants from named towns
    #by finding words that come between "купцы из" and "с", and removing those phrases that end in городов or части
    #it then separates out the individual towns, similar to merch_adj
    #merch_adj3 = re.findall(r"(?:купцам|Купцы|купцы|купцами|купцов|купечество)\s+из\s+((?:[а-яА-Я,]+\s+)*?)(?:с|;) ",txt)
    #merch_adj3 = [t.strip() for t in merch_adj3]
    #merch_adj3 = [t for t in merch_adj3 if len(re.findall(r"(?:городов|части)$",t))==0]
    #merch_adj3 = ru_lang_to_list(merch_adj3)
    #merch_adj.extend(merch_adj3)
    #merch_adj4 finds lists of towns from the style "from towns... came merchants", which occurs rarely
    merch_adj4 = re.findall(r"из\s+((?:(?!из)[А-Яа-я,]+\s+)*)(?:купцам|Купцы|купцы|купцами|купцов|купечество)",txt)
    merch_adj4 = ru_lang_to_list(merch_adj4)
    merch_adj4 = [re.sub(r" приезжают$","",a) for a in merch_adj4]
    merch_adj4 = [re.sub(r"^городов ","",a) for a in merch_adj4]
    if len(merch_adj4)!=0:
        merch_adj[4]=merch_adj4
    #merch_adj.extend(merch_adj4)
    if len(merch_adj) != 0:
        db[i]['merchants_from'] = merch_adj

In [94]:
from_counter = 0
for i in db:
    if 'merchants_from' in db[i]:
        from_counter+=1
        if verbose:
            print(db[i]['txt_id'],db[i]['merchants_from'])

In [95]:
from_counter

282

What goods are being traded?
============================
This section uses lists of specific and generic descriptions of goods being traded and adds them to the db dict.

In [96]:
with open("../resources/reverse_stem.json","r") as fp:
    reverse_stem = json.load(fp)

In [97]:
with open("../resources/goods_lookup.json","r",encoding="utf-8") as fp:
    goods_lookup = json.load(fp)

In [98]:
def goods_finder(text,good):
    """
    Finds `good` in `text` using reverse_stem dict loaded from json.
    Returns `True` if good is found, else returns `False`
    """
    if " " in good:
        if good in text:
            return True
        else:
            return False
    else:
        tokens = nltk.tokenize.wordpunct_tokenize(text)
        tokens = [t.strip(',.;:') for t in tokens]
        stem = ru_stemmer.stem(good)
        #print(reverse_stem[stem])
        if any([s in tokens for s in reverse_stem[stem]]):
            return True
        else:
            return False

In [99]:
#goodslist = ['скот','небогатый','гвоздь','рогожами','льном','ярмарми','дуга','веревка','материя','ладоном','домовными','мелсчными','горшками','заморский','полотно','сахар','серебряный','тутошний','продажный','сапог','число','недорогой','овца','произрастание','еъестными','конский','воск','вино','крамными','сковорода','сеестными','ром','шелк','неводный','гумми','медкими','скотский','покроми','иностранный','семь','сено','ском','пенечными','мед','один','бумажный','железный','хлеб','пушной','платками','особливый','коими','ливом','холст','позумент','сало','чулками','парчами','коса','шелковый','лыками','суровскими','вещми','мясами','продукт','вино','овчина','обыватель','серебреный','галантерейный','знатный','роза','выбойками','красный','припас','свой','пиво','набойчатыми','рост','съестной','уездный','колеса','купецкими','котлами','гостиный','мелочьми','лес','немалый','табак','чими','нитка','прочий','рукодельный','ными','китайка','купецкими','рукоделие','хлебный','говяжий','произрастениями','знатный','лавочный','варами','мелочми','брусья','питейный','обыкновенный','который','суконный','кожа','житель','масло','бечевами','ободами','холст','привозный','лошадьми','сукно','хомут','замками','дробязком','сосуд','оловянный','шерстяной','щепетильный','приуготовлениями','саньми','крестьянский','лес','какой','привозимый','холетом','москотильными','мелочь','деревянный','нужный','меховой','сани','щепетинными','внутренний','медный','гарусный','деревенский','железо','мелочный','крестьянами','приезжий','харчевыми','уклад','мелкий','купеческий','издельями','каждый','надлежащий']
spec_goodslist = ['ободами','саньми','сукно','выбойка','китайка','роза','лыками','коса','покроми','скотский','ладоном','дуга','скот','гвоздь','рогожами','льном','веревка','горшками','полотно','сахар','серебряный','сапог','овца','конский','воск','вино','сковорода','ром','шелк','гумми','сено','мед','хлеб','бумажный','железный','пушной','платками','холст','позумент','сало','чулками','парчами','шелковый','вино','овчина','серебреный','галантерейный','пиво','колеса','котлами','лес','табак','нитка','рукодельный','рукоделие','хлебный','говяжий','брусья','питейный','суконный','кожа','масло','бечевами','холст','лошадьми','хомут','замками','сосуд','оловянный','шерстяной','лес','меховой','сани','медный','гарусный','железо']
#adding multi-word goods separately for more convenient editing
spec_goodslist.extend(['красным товаром','Крымскую соль'])
kind_goodslist = ['произрастание','нужный','надлежащими','харчевыми','приуготовлениями','суровскими','набойчатыми','припасами','москотильными','обыватель','произрастениями','мясами','тутошний','небогатый','домовными','заморский','продажный','недорогой','крамными','медкими','иностранный','особливый','ливом','знатный','съестной','уездный','купецкими','немалый','ными','купецкими','знатный','лавочный','обыкновенный','житель','привозный','щепетильный','крестьянский','мелочь','деревянный','внутренний','щепетинными','деревенский','мелочный','крестьянами','приезжий','мелкий']

In [100]:
#set(kind_goodslist + spec_goodslist)

The next cell is where specific and general goods are extracted from the text and added to the dictionary db, which will ultimately become the dataset json file.

In [101]:
for i in db:
    sp_tempgoods = []
    gen_tempgoods = []
    for k,v in goods_lookup.items():
        try:
            if goods_finder(db[i]['description_new_orth'],k):
                if v['kind'] == 'specific':
                    sp_tempgoods.append(v['display'])
                elif v['kind'] == 'general':
                    gen_tempgoods.append(v['display'])
                else:
                    print("Kind not general or specific. Please edit kind key in json file.")
                    print(k,v)
            if len(sp_tempgoods)!=0:
                db[i]['specific_goods'] = sp_tempgoods
            if len(gen_tempgoods)!=0:
                db[i]['general_goods'] = gen_tempgoods
        except KeyError:
            print(good)

In [102]:
# Counts how many and can show which specific goods have been extracted.
sg_counter = 0
for i in db:
    if 'specific_goods' in db[i]:
        sg_counter+=1
        if verbose:
            print(db[i]['name_old_orth'],db[i]['specific_goods'])

In [103]:
sg_counter

287

Goods export
============
In this section the counts of what goods are traded where are collected and indexed by good, then by place, then exported to an Excel file.

In [104]:
goodscount = {}
for p in db:
    if 'specific_goods' in db[p]:
        for good in db[p]['specific_goods']:
            if good not in goodscount:
                goodscount[good]=1
            else:
                goodscount[good]+=1

In [105]:
spdf = pd.DataFrame.from_dict(goodscount,orient='index')
spdf.columns = ['Counts']
spdf = spdf.sort(columns='Counts',ascending=False)
spdf = spdf.fillna(0)

In [106]:
spdf.to_excel("sp_goods.xlsx",encoding='utf-8')
spdf.to_csv("sp_goods.csv",encoding='utf-8')

for i in db:
    tempgoods = []
    for good in kind_goodslist:
        try:
            if goods_finder(db[i]['description_new_orth'],ru_stemmer.stem(good)):
                if good in equiv_dict:
                    good = equiv_dict[good]
                tempgoods.append(good)
            if len(tempgoods)!=0:
                db[i]['general_goods'] = tempgoods
        except KeyError:
            print(good)

In [107]:
kg_counter = 0
for i in db:
    if 'general_goods' in db[i]:
        kg_counter+=1
        if verbose:
            print(db[i]['name_old_orth'],db[i]['general_goods'])

In [108]:
kg_counter

477

In [109]:
general_goodscount = {}
for p in db:
    if 'general_goods' in db[p]:
        for good in db[p]['general_goods']:
            if good not in general_goodscount:
                general_goodscount[good]=1
            else:
                general_goodscount[good]+=1

In [110]:
gdf = pd.DataFrame.from_dict(general_goodscount,orient='index')
gdf.columns = ['Counts']
gdf = gdf.sort(columns='Counts',ascending=False)
gdf = gdf.fillna(0)

In [111]:
gdf.to_excel("gen_goods.xlsx",encoding='utf-8')

In [112]:
with open("../resources/partof_prov_bounds.json","r",encoding='utf-8') as fp:
    partof_lookup = json.load(fp)

In [113]:
prov_sp_goods = {}
for i in db:
    if 'specific_goods' in db[i]:
        if not pd.isnull(db[i]['admin1_partofID']) and db[i]['admin1_partofID'] != 'нот_ин_теxт':
            partof_name = partof_lookup[str(float(db[i]['admin1_partofID']))]['name']
        else:
            partof_name = 'None'
        if partof_name not in prov_sp_goods:
            prov_sp_goods[partof_name] = {}
        for good in db[i]['specific_goods']:
            if good not in prov_sp_goods[partof_name]:
                prov_sp_goods[partof_name][good] = 1
            else:
                prov_sp_goods[partof_name][good]+=1

In [114]:
sp_prov_df = pd.DataFrame.from_dict(prov_sp_goods)
sp_prov_df = sp_prov_df[list(sp_prov_df.columns[::-1])]
sp_prov_df['Total'] = sp_prov_df.sum(axis=1)
sp_prov_df = sp_prov_df.sort(columns='Total',ascending=False)
sp_prov_df = sp_prov_df.fillna(0)

In [115]:
sp_prov_df.to_excel('sp_by_prov.xlsx',encoding='utf-8')
sp_prov_df.to_csv('sp_by_prov.csv',encoding='utf-8')

In [116]:
prov_k_goods = {}
for i in db:
    if 'general_goods' in db[i]:
        if not pd.isnull(db[i]['admin1_partofID']) and db[i]['admin1_partofID'] != 'нот_ин_теxт':
            partof_name = partof_lookup[str(float(db[i]['admin1_partofID']))]['name']
        else:
            partof_name = 'None'
        if partof_name not in prov_k_goods:
            prov_k_goods[partof_name] = {}
        for good in db[i]['general_goods']:
            if good not in prov_k_goods[partof_name]:
                prov_k_goods[partof_name][good] = 1
            else:
                prov_k_goods[partof_name][good]+=1

In [117]:
gen_prov_df = pd.DataFrame.from_dict(prov_k_goods)
gen_prov_df = gen_prov_df[gen_prov_df.columns[::-1]]
gen_prov_df['Total'] = gen_prov_df.sum(axis=1)
gen_prov_df = gen_prov_df.sort(columns='Total',ascending=False)
gen_prov_df = gen_prov_df.fillna(0)

In [118]:
gen_prov_df.to_excel('gen_by_prov.xlsx',encoding='utf-8')
gen_prov_df.to_csv('gen_by_prov.csv',encoding='utf-8')

In [119]:
all_goods_count = 0
for k,v in db.items():
    if 'general_goods' in v or 'specific_goods' in v:
        all_goods_count += 1
print(all_goods_count)

507


When are the markets?
=====================
Here we are looking for dates of markets, mostly explicit, calendar dates, but also dates that are implicit in religious holidays.

In [120]:
import datetime
def txt_to_datetime(month,day,year=1788):
    monthdict = {'Генваря месяца':1,"Генваря":1,"Февраля":2,"Марта":3,"Апреля":4,"Мая":5,"Июня":6,"Июля":7,"Августа":8,"Сентября":9,"Октября":10,"Ноября":11,"Декабря":12,'Генваре':1,'Феврале':2,'Марте':3,'Апреле':4,'Маие':5,'Июне':6,'Июле':7,'Августе':8,'Сентябре':9,'Октябре':10,'Ноябре':11,'Декабре':12}
    m = monthdict[month]
    d = int(day)
    date = datetime.date(year,m,d).strftime("%Y-%m-%d")
    return date

In [121]:
def date_range_finder(text,year=1788):
    """
    Takes a text entry and uses a series of regexes to find start and end dates in given text
    """
    monthlist = ["Генваря","Февраля","Марта","Апреля","Мая","Июня","Июля","Августа","Сентября","Октября","Ноября","Декабря"]
    remonth = "|".join(monthlist)
    ranges = []
    range1 = re.findall(r"(с\s+([0-9]+)\s+({})\s+по\s+([0-9]+)\s+({}))".format(remonth,remonth),text)
    if len(range1)!=0:
        for r in range1:
            temp = {'beg':txt_to_datetime(r[2],r[1]),'end':txt_to_datetime(r[4],r[3]),'text':r[0],'paschal':False}
            ranges.append(temp)
    range2 = re.findall(r"(({})\s+с\s+([0-9]+)\s+({})\s+по\s+([0-9]+))".format(remonth,remonth),text)
    if len(range2)!=0:
        for r in range2:
            temp = {'beg':txt_to_datetime(r[1],r[2]),'end':txt_to_datetime(r[3],r[4]),'text':r[0],'paschal':False}
            ranges.append(temp)
    range3 = re.findall(r"(({})\s+от\s+([0-9]+)\s+по\s+([0-9]+))".format(remonth,remonth),text)
    if len(range3)!=0:
        for r in range3:
            temp = {'beg':txt_to_datetime(r[1],r[2]),'end':txt_to_datetime(r[1],r[3]),'text':r[0],'paschal':False}
            ranges.append(temp)
    return ranges

In [122]:
def single_date_finder(text):
    """
    Finds single dates using various regexes and returns them as a list of date ranges
    """
    monthlist = ["Генваря","Февраля","Марта","Апреля","Мая","Июня","Июля","Августа","Сентября","Октября","Ноября","Декабря"]
    remonth = "|".join(monthlist)
    ranges = []
    dates1 = re.findall(r"(([0-9]+)\s+(?:числа|день)?(?:\s+)?({}))".format(remonth),text)
    if len(dates1)!=0:
        for r in dates1:
            test = re.findall(r"([а-яА-Я0-9:,]+)\s+([а-яА-Я0-9:,]+)\s+{}".format(r[0]),text)
            if test[0] in monthlist and test[1] == 'с':
                pass
            else:
                temp = {'beg':txt_to_datetime(r[2],r[1]),'end':txt_to_datetime(r[2],r[1]),'text':r[0],'paschal':False}
                ranges.append(temp)
    dates2 = re.findall(r"(({})\s+(?:по|от|на|с|в|месяца в)?(?:\s+)?([0-9]+))".format(remonth),text)
    if len(dates2)!=0:
        for r in dates2:
            temp = {'beg':txt_to_datetime(r[1],r[2]),'end':txt_to_datetime(r[1],r[2]),'text':r[0],'paschal':False}
            ranges.append(temp)
    return(ranges)

In [123]:
import calendar
def month_finder(text,year=1788):
    """
    Finds mentions of months in nominative form, returns date ranges for the whole month
    """
    nom_monthdict = {'Генваря месяца':1,'Генваре':1,'Феврале':2,'Марте':3,'Апреле':4,'Маие':5,'Июне':6,'Июле':7,'Августе':8,'Сентябре':9,'Октябре':10,'Ноябре':11,'Декабре':12}
    months = re.findall(r"({})".format("|".join(nom_monthdict.keys())),text)
    ranges = []
    for m in months:
        temp = {'beg':txt_to_datetime(m,1),'end':txt_to_datetime(m,calendar.monthrange(year,nom_monthdict[m])[1]),'text':m,'paschal':False}
        ranges.append(temp)
    return ranges

In [124]:
for i in db:
    if 'fair_dates' in db[i]:
        db[i].pop('fair_dates')
    ranges = date_range_finder(db[i]['description_new_orth'])
    ranges1 = single_date_finder(db[i]['description_new_orth'])
    if len(ranges)!=0 and len(ranges1)!=0:
        to_rm = []
        for ind1 in range(0,len(ranges1)):
            for ind in range(0,len(ranges)):
                if ranges[ind]['beg'] == ranges1[ind1]['beg']:
                    to_rm.append(ranges1[ind1])
                elif ranges[ind]['end'] == ranges1[ind1]['end']:
                    to_rm.append(ranges1[ind1])
        for p in to_rm:
            ranges1.remove(p)
    ranges.extend(ranges1)
    ranges2 = month_finder(db[i]['description_new_orth'])
    ranges.extend(ranges2)
    if len(ranges)!=0:
        db[i]['fair_dates']=ranges
        if verbose:
            print(db[i]['description_new_orth'])
            for r in db[i]['fair_dates']:
                if r['beg']==r['end']:
                    print(r['end'])
                else:
                    print("{} to {}".format(r['beg'],r['end']))
            print()

In [125]:
easter = datetime.date(1788,4,16) #This is in a Julian calendar, which is 11 days behind the Gregorian

def movable_feast(name,easter=datetime.date(1788,4,16)):
    """
    Takes name of feast, looks it up in stored lookup dict, returns date range relative to given date of easter.
    Always returns dict with beginning and end, even for single day holidays.
    """
    with open("../holidays.json","r",encoding='utf-8') as fp:
        lookup = json.load(fp)
    if name in lookup:
        offset = lookup[name]
        if type(offset) == int:
            date = easter + datetime.timedelta(days=offset)
            date = date.strftime("%Y-%m-%d")
            val = {'beg':date,'end':date,'text':name}
        elif type(offset) == list:
            date1 = easter + datetime.timedelta(days=offset[0])
            date1 = date1.strftime("%Y-%m-%d")
            date2 = easter + datetime.timedelta(days=offset[1])
            date2 = date2.strftime("%Y-%m-%d")
            val = {'beg':date1,'end':date2,'text':name}
        else:
            print("The value of {} in the lookup dict is neither list nor int. Fix plz.".format(name))
            val = None
    else:
        print("\"{}\" was not found in the lookup dict. Add it and rerun.".format(name))
        val = None
    return val

In [126]:
with open("../resources/holiday_regex.json","r",encoding='utf-8') as fp:
    holiday = json.load(fp)

In [127]:
easter = datetime.date(1788,4,16)
for k,v in db.items():
    for n,i in holiday.items():
        exps = i['expression']
        t = v['description_new_orth']
        for r in exps:
            found = re.findall(r,t,re.IGNORECASE)
            if len(found)!=0:
                temp = {"text":"|".join(found)}
                dates = i['dates']
                if type(dates)==str:
                    temp['beg'] = "1788-{}".format(dates)
                    temp['end'] = "1788-{}".format(dates)
                    temp['paschal'] = False
                elif type(dates) == int:
                    temp['beg'] = (easter + datetime.timedelta(dates)).strftime("%Y-%m-%d")
                    temp['end'] = (easter + datetime.timedelta(dates)).strftime("%Y-%m-%d")
                    temp['paschal'] = True
                elif type(dates) == list:
                    if type(dates[0])==str:
                        temp['beg'] = "1788-{}".format(dates[0])
                        temp['end'] = "1788-{}".format(dates[1])
                        temp['paschal'] = False
                    elif type(dates[0])==int:
                        temp['beg'] = (easter + datetime.timedelta(dates[0])).strftime("%Y-%m-%d")
                        temp['end'] = (easter + datetime.timedelta(dates[1])).strftime("%Y-%m-%d")
                        temp['paschal'] = True
                    else:
                        print("These dates are a list of {}, when they should be int or str. Fix plz.\n{} at holiday {}".format(type(dates[0]),dates,n))
                else:
                    print("These dates are of  type {}, when they should be int, str, or list. Fix plz.\n{} at holiday {}".format(type(dates[0]),dates,n))
                if 'fair_dates' not in v:
                    db[k]['fair_dates'] = []
                db[k]['fair_dates'].append(temp)

In [128]:
def date_overlap(date_list1,date_list2,string=True):
    """
    Checks if beginning and end dates in date_list1 overlap with those of date_list2
    The string flag indicates whether datetime objects or ISO 8601 date strings are being passed
    """
    if string:
        date_list1 = [datetime.datetime.strptime(s,"%Y-%m-%d") for s in date_list1]
        date_list2 = [datetime.datetime.strptime(s,"%Y-%m-%d") for s in date_list2]
    if date_list1[0]>=date_list2[0] and date_list1[0]<=date_list2[1]:
        return True
    elif date_list1[1]>=date_list2[0] and date_list1[1]<=date_list2[1]:
        return True
    else:
        return False

In [129]:
def overlap_check(fair_date_list):
    """
    Takes a list of fair dates from data structure as input. Checks if any dates overlap, and if they do, selects the
    date with the shortest text justifying it to be removed.
    Returns a list of indices to be removed from list of fair date dictionaires.
    """
    to_remove = []
    for x, left in enumerate(fair_date_list):
        for y, right in enumerate(fair_date_list):
            if x==y:
                continue
            else:
                temp1 = [left['beg'],left['end']]
                temp2 = [right['beg'],right['end']]
                if date_overlap(temp1,temp2):
                    if len(re.findall(r"[0-9]+",left['text']))!=0:
                        to_remove.append(y)
                    elif len(re.findall(r"[0-9]+",right['text']))!=0:
                        to_remove.append(x)
                    elif len(left['text']) > len(right['text']):
                        to_remove.append(y)
                    elif len(left['text']) < len(right['text']):
                        to_remove.append(x)
                    else:
                        print("Why do the dates for {} and {} overlap? The phrases are the same length.".format(left['text'],right['text']))
    to_remove = list(set(to_remove))
    to_remove.sort(reverse=True)
    return to_remove

In [130]:
for k,v in db.items():
    if 'fair_dates' in v:
        o = overlap_check(v['fair_dates'])
        if len(o)>0:
            for i in o:
                v['fair_dates'].pop(i)
                if verbose:
                    print("{} removed.".format(v['fair_dates'][i]))
                    print(v['description_new_orth'])
                    print(v['fair_dates'])
                    print()

In [131]:
for k,v in db.items():
    if 'fair_dates' in v and len(v['fair_dates'])==0:
        print(v)

In [132]:
for k,v in db.items():
    if len(re.findall(r"четыре ярмарки",v['description_new_orth']))!=0 and 'fair_dates' in db[k] and len(v['fair_dates'])!=4:
        print("{}: {}".format(k,v['description_new_orth']))
        for d in v['fair_dates']:
            if d['beg']==d['end']:
                print("{}\t\t\ttext: {}".format(d['beg'],d['text']))
            else:
                print("{} to {}\ttext: {}".format(d['beg'],d['end'],d['text']))
        print()

28: АХТЫРКА , город Харьковского Наместничества; в нем бывает четыре ярмарки: первая о всеедной неделе, вторая Мая 9, третия Июля четвертая Октября 28 дня; купцы приезжают из раз ных городов, привозят парчи , шелковые материи и серебро в деле; торгуют также скотом , лошадьми и прочим.
1788-05-09			text: Мая 9
1788-10-28			text: Октября 28
1788-02-06 to 1788-02-12	text: всеедной неделе

584: РОМЕН, город Черниговского Наместничества. В оном бывает в год четыре ярмарки : первая о Масляной неделе, вторая в день Вознесения , третия на день Пророка Илии, четвертая в день Собора Архистратига Михаила. Сюда приезжают купцы Великороссийские , Малороссийские и иностранные с шелковыми, бумажными , гарусными , серебреными , медными , оловянными, галантерейными и прочими товарами , а особливо великой торг и отпуск бывает отсюда табаку.
1788-05-25			text: Вознесения
1788-11-08			text: Архистратига Михаила
1788-11-21			text: день Собора Архистратига Михаила
1788-02-21 to 1788-02-27	text: Масляной
178

In [133]:
temp = []
for k,v in db.items():
    if 'fair_dates' not in db[k]:
        temp.append([k,len(v['description_new_orth']),v['description_new_orth']])

In [134]:
pd.DataFrame(temp).to_csv("no_fair.csv",encoding='utf-8')

In [135]:
counter = 0
for k,v in db.items():
    if 'fair_dates' in v and any(s in v for s in ['specific_goods','general_goods']) and 'geo' not in v:
        if verbose:
            print(k, v['description_new_orth'])
            print()
        counter += 1

In [136]:
counter

139

In [137]:
counter = 0
for k,v in db.items():
    if 'fair_dates' in v and any(s in v for s in ['specific_goods','general_goods']) and 'geo' in v:
        counter += 1
counter

237

In [138]:
temp = []
for k,v in db.items():
    if 'geo' not in v:
        temp.append([k,len(v['description_new_orth']),v['description_new_orth'],v['admin1_old_orth'],v['admin2_old_orth'],v['featuretype_old_orth']])
long_not_found = pd.DataFrame(temp).sort(columns=1,ascending=False)

In [139]:
long_not_found.to_csv("../output/process/long_not_found.csv",encoding='utf-8')

In [140]:
db['281']['description_new_orth']

'КИТАЙСКОЙ, острог в Екатеринбургском Горном ведомстве. Здесь напред сего бывала великая ярмарка на день Богоявления Господня, на которую съезжалось множество купцов из Сибири и из Россия , а особливо из Казани чрез Уфу по тому, что тогда большая дорога из Казани в Сибирь была чрез Уфу и чрез оной Катайской острог; но с построения Екатеринбурга сия Катайская ярмарка пресеклась по тому, что уже большая дорога проложена чрез сей город.'

In [141]:
dates_counter = 0
for k,v in db.items():
    if 'fair_dates' in v:
        dates_counter +=1
        if verbose:
            print(v['description_new_orth'])
            for d in v['fair_dates']:
                if d['beg']==d['end']:
                    print("{}\t\t\ttext: {}".format(d['beg'],d['text']))
                else:
                    print("{} to {}\ttext: {}".format(d['beg'],d['end'],d['text']))
            print()

In [142]:
dates_counter

525

In [143]:
def juli_to_greg(date,string=True,offset=11):
    """
    Converts ISO-8601 date string from Julian calendar format to current, Gregorian Date
    Set string to false if providing datetime.date objects
    offset is how many days to adjust Julian Calendar forward.
    """
    if string:
        date = datetime.datetime.strptime(date,"%Y-%m-%d")
    greg_date = date + datetime.timedelta(offset)
    if greg_date.year != date.year:
        greg_date = datetime.datetime.strptime("{}-{}".format(date.year,greg_date.strftime("%m-%d")),"%Y-%m-%d")
    if string:
        return greg_date.strftime("%Y-%m-%d")
    else:
        return greg_date

In [144]:
from dateutil import rrule

In [145]:
weekdays1788 = [[],[],[],[],[],[],[]]
for r in rrule.rrule(rrule.DAILY,dtstart=datetime.datetime(1788,1,1),until=datetime.datetime(1788,12,31)):
    d = juli_to_greg(r,string=False)
    lookup = {0:"monday",1:"tuesday",2:"wednesday",3:"thursday",4:"friday",5:"saturday",6:"sunday"}
    weekdays1788[d.weekday()].append(r.strftime("%Y-%m-%d"))

In [146]:
def alternify(word):
    """Not easily movable. Returns a list of all alternate forms of a word based on its stemmed form and all other words appearing 
    in the text that stem to the same form."""
    alternates = reverse_stem[ru_stemmer.stem(word)]
    return alternates

In [147]:
def words_in_sent(wordlist,sent_str,alter=True):
    if alter:
        wordlist = ["|".join(alternify(w)) for w in wordlist]
    if all([len(re.findall(r"(?:{})".format(w),sent_str))!=0 for w in wordlist]):
        return True
    else:
        return False

In [148]:
def every_weekday(ru_lookup_day,weekday_int,text,verbose=False):
    for part in re.split(r"[:;]",text):
        for s in nltk.tokenize.sent_tokenize(part):
            if words_in_sent([ru_lookup_day,"еженедельные"],s):
                if verbose:
                    print(s)
                return weekdays1788[weekday_int]
            elif words_in_sent([ru_lookup_day,"каждонедельно"],s):
                if verbose:
                    print(s)
                return weekdays1788[weekday_int]
    return None

In [149]:
def date_to_entry(date,text,string=True,weekly=False):
    if not string:
        date = date.strftime("%Y-%m-%d")
    entry = {}
    entry['beg'] = date
    entry['end'] = date
    entry['text'] = text
    if weekly:
        entry['weekly'] = True
    return entry

In [150]:
trade_count = 0
ru_weekdays = [("пятницам",4),('понедельник',0),('пяток',4),('Воскресяньям',6),('Воскресеньям',6),('вторник',1),('среда',2),('четверг',3),('суббота',5)]
for k,v in db.items():
    found = False
    for d in ru_weekdays:
        wkd = every_weekday(d[0],d[1],v['description_new_orth'])
        if wkd:
            found=True
            #if 'fair_dates' not in v:
                #v['fair_dates'] = []
            for day in wkd:
                #v['fair_dates'].append(date_to_entry(day,"еженедельные {}".format(d[0]),weekly=True))
                pass
            if verbose:
                print(v['description_new_orth'])
                print(d[0])
                print()
    if found:
        trade_count+=1

In [151]:
trade_count

43

In [152]:
all_goods = 0
for k,v in db.items():
    if 'specific_goods' in v or 'general_goods' in v:
        all_goods+=1

In [153]:
goods_and_dates = 0
for k,v in db.items():
    if ('specific_goods' in v or 'general_goods' in v) and 'fair_dates' in v:
        goods_and_dates+=1

In [154]:
dates_counter = 0
for k,v in db.items():
    if 'fair_dates' in v:
        dates_counter+=1

In [155]:
print("Out of {} total entries:\n{} have some description of where goods are coming from / going to\n{} have descriptions of specific traded goods\n{} have descriptions of general kinds of traded goods\n{} have descriptions of goods of any kind\n{} have specific dates associated with the fairs\n{} have both fair dates and some description of goods".format(len(db),from_counter,sg_counter,kg_counter,all_goods,dates_counter,goods_and_dates))

Out of 802 total entries:
282 have some description of where goods are coming from / going to
287 have descriptions of specific traded goods
477 have descriptions of general kinds of traded goods
507 have descriptions of goods of any kind
525 have specific dates associated with the fairs
376 have both fair dates and some description of goods


In [156]:
db['539']['description_old_orth']

'ПЛОХОНА, село Калужскаго Намѣстничества въ уѣздѣ города Козельска. Здѣсь бываютъ каждонедѣльные торжки по воскресеньямъ; на оные торги пріѣзжаетъ купечество изъ Калуги, Мещовска, Волхова, Бѣлева и другихъ городовъ; привозятъ товары москотильные, нѣсколько бумажныхъ, шелковыхъ и другихъ сортовъ матерій , а по большей части и уѣздные товары, всякой хлѣбъ, пеньку, масло , ленъ , воскъ , вощину , парусную уточную пряжу, холстъ льняной, посконной и прочія крестьянскія рукодѣлія и другіе мѣлочные товары.'

for e in db:
    temp_goodslist = word_case_sent_finder(['приезжают','привозят','торгуют','съежаются','скупают','покупают'],r"[а-яА-Я]+(?:ми|ом)$",db[e]['description_new_orth'])
    goodsfrom = [g for g in temp_goodslist if re.match(r"^[А-Я]",g)]
    goodskinds = [g for g in temp_goodslist if re.match(r"^[а-я]",g)]
    removelist = ['купеческими','издельями','каждом','какими','привозимыми','котором','чими','своими','продуктами','коими','пенечными','семи','оном','товарами','товаром','вещами','всякими','другими','другом','ими','различными','разными','торгом','том','числом','неводными']
    for i in removelist:
        if i in goodskinds:
            goodskinds.remove(i)
    if len(goodsfrom)!=0:
        db[e]['goodsfrom']=goodsfrom
    if len(goodskinds)!=0:
        db[e]['goodskinds'] = goodskinds

counter=0
for e in db:
    if 'goodskinds' in db[e]:
        print(db[e]['goodskinds'])
        counter += 1

In [157]:
#counter

In [158]:
with open("../output/enhanced_dataset.json","w",encoding='utf-8') as fp:
    json.dump(db,fp,indent=2,sort_keys=True)

In [159]:
geojson = {"features":[],"type":"FeatureCollection"}
for k,v in db.items():
    if 'geo' in v:
        feature = {"geometry":{'coordinates':[v['geo']['x_coord'],v['geo']['y_coord']],"type":"Point"},"type":"Feature"}
        feature['properties'] = v
        geojson['features'].append(feature)

In [160]:
with open("../output/enhanced_dataset.geojson","w",encoding='utf-8') as fp:
    json.dump(geojson,fp,sort_keys=True)

In [161]:
txt_counter = 0
for k,v in db.items():
    print("{}|{}".format(k,v['description_new_orth']))

123|ВОРОНЕЖ, главный город Наместничества сего имени. В нем, кроме еженедельных по понедельникам , середам и пятницам торгов , бывают в году две ярмарки: первая в городе о десятой пятнице , а последняя при слободе Чижовке в 29 день Августа.
60|БОЙНЯ, река Тверского Наместничества. На устье оной в уезде города Зубцова бывают в году две ярмарки: первая 20 Июля , в день Пророка Илии, вторая Августа 6, в день Преображения. Купцы приезжают из окрестных ближних городов с мелочными лавочными шелковыми , бумажными и прочими товарами.
557|ПОШЕХОНЬ, город Ярославского Наместничества. Здесь , кроме еженедельного торгу по вторникам , ярмарка бывает 1 на праздник Святые Троицы , 2 в Сентябре на праздник Иоанна Богослова, на которые приезжают купцы из прикосновенных городов , Ярославля , Романова , Углича и Вологды с разными товарами , и торгуют по неделе.
470|НОВОЕЗЕРСКОЙ КИРИЛОВ, монастырь Новогородского Наместничества в уезде города Белозерска. Здесь бывает годовая ярмарка Марта 17, называемая Ал

In [162]:
with open("../resources/machine_translated.json","r",encoding='utf-8') as fp:
    translate = json.load(fp)
for k in db:
    db[k]['desc_machine_translated'] = translate[k]

In [163]:
keys = {}
for k,v in db.items():
    for j in v:
        keys[j]="explanation"
        if type(v[j])==dict:
            for h in v[j]:
                keys[j+"_"+str(h)] = 'explanation'

Calendar Presentation
=====================

In [164]:
year = [html.fromstring(calendar.HTMLCalendar(firstweekday=6).formatmonth(1788,m)) for m in range(1,13)]

In [165]:
for m in range(len(year)):
    for c in year[m].getchildren():
        for cc in c.getchildren():
            try:
                i = int(cc.text_content())
                cc.set("id","1788-{:0>2}-{:0>2}".format(m+1,i))
            except ValueError:
                continue

In [166]:
datedict = {}
for k,v in db.items():
    if 'fair_dates' in v:
        for d in v['fair_dates']:
            beg = datetime.datetime.strptime(juli_to_greg(d['beg']),"%Y-%m-%d")
            end = datetime.datetime.strptime(juli_to_greg(d['end']),"%Y-%m-%d")
            while beg <= end:
                f_beg = beg.strftime("%Y-%m-%d")
                if f_beg not in datedict:
                    datedict[f_beg] = []
                datedict[f_beg].append("{}: {}".format(v['name_new_orth'],d['text']))
                beg = beg + datetime.timedelta(1)

In [167]:
for d,v in datedict.items():
    m = int(d[5:7])-1
    to_add = html.fromstring("<ul><li>{}</li></ul>".format("</li><li>".join(v)))
    year[m].get_element_by_id(d).extend(to_add)

In [168]:
from IPython.display import HTML

In [169]:
HTML(str(html.tostring(year[0]).replace(b"\n",b"")))

In [170]:
with open("../fair_calendar.html","w+",encoding='utf-8') as fp:
    for m in year:
        fp.write(html.tostring(m).decode('utf-8'))

In [171]:
with open("../output/enhanced_dataset.geojson","r",encoding='utf-8') as fp:
    geodb = json.load(fp)

In [172]:
for f in geodb['features']:
    if 'fair_dates' in f['properties']:
        for d in f['properties']['fair_dates']:
            name = f['properties']['name_new_orth']
            lng = f['geometry']['coordinates'][0]
            lat = f['geometry']['coordinates'][1]
            beg = juli_to_greg(d['beg'])
            end = juli_to_greg(d['end'])
            try:
                print("{}\t{}\t{}\t{}\t{}\t{}\t{}".format(name,lng,lat,beg,end,d['text'],d['paschal']))
            except KeyError:
                print(f['properties']['name_old_orth'],d)

Воронеж	39.210556	51.671667	1788-09-09	1788-09-09	29 день Августа	False
Воронеж	39.210556	51.671667	1788-07-04	1788-07-04	десятой пятнице	True
Пошехонь	39.13531	58.49928	1788-09-12	1788-10-11	Сентябре	False
Пошехонь	39.13531	58.49928	1788-05-19	1788-05-19	Иоанна Богослова	False
Пошехонь	39.13531	58.49928	1788-06-22	1788-06-22	Троицы	True
Остапье	33.76977	49.54487	1788-02-10	1788-02-10	Генваря 30	False
Остапье	33.76977	49.54487	1788-05-02	1788-05-02	Апреля 21	False
Остапье	33.76977	49.54487	1788-07-05	1788-07-05	Июня 24	False
Остапье	33.76977	49.54487	1788-10-01	1788-10-01	Сентября 20	False
Углич	38.33167	57.5275	1788-07-19	1788-07-19	Июля 8	False
Углич	38.33167	57.5275	1788-01-06	1788-01-06	Декабря 26	False
Вышгород	29.62476	57.48135	1788-07-05	1788-07-05	Иоанна Предтечи	False
Вышгород	29.62476	57.48135	1788-08-04	1788-08-04	Бориса и Глеба	False
Сердоболь	30.67434	61.69459	1788-01-12	1788-02-11	Генваре	False
Гремячев	38.6607	54.13149	1788-06-23	1788-06-23	первый день Петрова поста	True

In [173]:
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = str(parent_key) + sep + str(k) if parent_key else str(k)
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            if type(v) == list:
                for i,lv in enumerate(v):
                    if type(lv)!=dict:
                        items.append((new_key + sep + str(lv),1))
                    else:
                        items.extend(flatten(i, new_key, sep=sep).items())
            else:
                items.append((new_key, v))
    return dict(items)

In [174]:
test2 = {"a":{"1":1,"2":[{'3':12},{'3':14}]},"b":4}

In [175]:
flatten(test2)

AttributeError: 'int' object has no attribute 'items'

In [ ]:
flat_db = {}
for k,v in db.items():
    flat_db[k]=flatten(v)

In [ ]:
flat_df = pd.DataFrame.from_dict(flat_db,orient='index')

In [ ]:
flat_df.to_excel("../output/flat_dataset.xlsx",encoding='utf-8')

In [ ]:
db_bygood = []
for k,v in db.items():
    if 'specific_goods' in v and 'fair_dates' in v:
        for g in v['specific_goods']:
            for d in v['fair_dates']:
                temp = {}
                temp['good'] = g
                temp['txt_id'] = k
                for d1 in d:
                    temp["fair_"+d1]=d[d1]
                for i in v:
                    if type(v[i])!=list and type(v[i])!=dict:
                        temp[i] = v[i]
                if 'geo' in v:
                    for h in v['geo']:
                        temp["geo_"+h] = v['geo'][h]
                db_bygood.append(temp)

In [ ]:
df_bygood = pd.DataFrame(db_bygood)

In [ ]:
temp = []
for k,v in db.items():
    if 'specific_goods' in v and 'fair_dates' in v:
        temp.append(k)
len(set(temp))

In [ ]:
txt_ids = [str(i) for i in list(df_bygood.txt_id.unique())]
for i in temp:
    if i not in txt_ids:
        if len(db[str(i)]['fair_dates'])==0:
            print("confirmation")
        else:
            print("{} is an exception to my hypothesis".format(i))

In [ ]:
len(txt_ids)

In [ ]:
df_bygood[df_bygood.txt_id==549]

In [ ]:
db['549']

In [ ]:
df_bygood.to_excel("../output/dataset_by_goods.xlsx",encoding='utf-8')